<h1><center>ETL / Desarrollo de funciones </center></h1>

<h1> Importación de librerías </h1>

In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime 

<h1>Cargar datasets JSON </h1>

In [2]:
#cargar reviews
rows=[]

with open('australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_games_reviews=pd.DataFrame(rows)
df_games_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [3]:
rows=[]

with open('australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_games_items=pd.DataFrame(rows)
df_games_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [4]:
df_steam_games=pd.read_json('output_steam_games.json',lines=True)
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


<h1>Preparación de los datos </h1>

In [5]:
#se va a preparar, el df de itemsgames
df_games_items.head(2)


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [6]:
print(df_games_items.shape)
df_games_items.info()

(88310, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
#se va a preparar games reviews
df_games_reviews.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [8]:
print(df_games_reviews.shape)
df_games_reviews.info()

(25799, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [9]:
print(df_steam_games.shape)
df_steam_games.info()

(120445, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [10]:
df_steam_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [11]:
df_steam_games.dropna(inplace=True)
#se cambia el tipo de dato para poder efectuar posteriormente las consultas
df_steam_games['id'] = df_steam_games['id'].astype(int)
df_steam_games['id'] = df_steam_games['id'].astype(str)

In [12]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540,Trickjump Games Ltd


In [13]:
#se reemplazará 'free to play' a 0, ya que no tiene costo.
df_steam_games['price'] = df_steam_games['price'].apply(lambda x: float(x) if isinstance(x, (int, float)) else 0)

In [14]:
df_steam_games['price'] = df_steam_games['price'].astype(float)

In [15]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   app_name      22530 non-null  object 
 3   title         22530 non-null  object 
 4   url           22530 non-null  object 
 5   release_date  22530 non-null  object 
 6   tags          22530 non-null  object 
 7   reviews_url   22530 non-null  object 
 8   specs         22530 non-null  object 
 9   price         22530 non-null  float64
 10  early_access  22530 non-null  float64
 11  id            22530 non-null  object 
 12  developer     22530 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.4+ MB


In [16]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540,Trickjump Games Ltd


In [17]:
df_games_reviews.dropna(inplace=True)
df_games_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [18]:
#expansión del df reviews
expanded_df_reviews = df_games_reviews.explode('reviews', ignore_index=True)

In [19]:
expanded_df_reviews = pd.concat([expanded_df_reviews, expanded_df_reviews['reviews'].apply(pd.Series)], axis=1)

C:\Users\Luis Mario Gomez\AppData\Local\Temp\ipykernel_35316\3234143580.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  expanded_df_reviews = pd.concat([expanded_df_reviews, expanded_df_reviews['reviews'].apply(pd.Series)], axis=1)


In [20]:
expanded_df_reviews.drop(columns='reviews', inplace=True)

In [21]:
expanded_df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [22]:
data_desanidada = []

for index, row in df_games_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        data_desanidada.append(new_row)

In [23]:
expanded_df_items = pd.DataFrame(data_desanidada)

In [24]:
#se borrarán las reviews con fechas sin año
# Eliminar filas con valores NaN en la columna 'posted'
expanded_df_reviews = expanded_df_reviews.dropna(subset=['posted'])

In [25]:
filtered_reviews = expanded_df_reviews[expanded_df_reviews['posted'].str.contains(r'\d{4}')]
expanded_df_reviews = filtered_reviews.copy()

In [26]:
#se trabajarán las fechas de las reviews
def convert_date_format(date_str:str):
    date_str = date_str.strip('Posted ')
    date_str = date_str.strip('.')
    date_format = "%B %d, %Y"
    date = datetime.strptime(date_str, date_format)
    return date

expanded_df_reviews['posted'] = expanded_df_reviews['posted'].apply(convert_date_format)

<h1>Feature engineering </h1>

In [27]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to C:\Users\Luis Mario
[nltk_data]     Gomez\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
def assign_sentiment(row):
    if pd.isna(row['review']):
        return 1
    sentiment_score = sia.polarity_scores(row['review'])['compound']
    if sentiment_score >= 0.05:
        return 2
    elif sentiment_score >= 0.05:
        return 0
    else:  # Neutral
        return 1 

In [29]:
expanded_df_reviews['sentiment_analysis'] = expanded_df_reviews.apply(assign_sentiment,axis=1)

In [30]:
expanded_df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN,2
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN,2


In [31]:
count_positive_sentiments = expanded_df_reviews['sentiment_analysis'].value_counts().get(2, 0)
print("Número de celdas con valor 2 en sentiment_analysis:", count_positive_sentiments)
expanded_df_reviews.shape

Número de celdas con valor 2 en sentiment_analysis: 31392


(49186, 11)

In [32]:
expanded_df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN,2
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN,2


<h1>Desarrollo API </h1>

In [33]:
#para este punto nuestros df serán
#expanded_df_items
#expanded_df_reviews
#df_steam_games.head


In [34]:
#Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items
def userdata(User_id: str):
    user_games = expanded_df_items[expanded_df_items['user_id'] == User_id]['item_id']
    prices = df_steam_games[df_steam_games['id'].isin(user_games)]['price']
    money_spent_by_user = prices.sum()
    
    user_reviews = expanded_df_reviews[expanded_df_reviews['user_id'] == User_id]
    total_reviews = user_reviews.shape[0]
    recommended_reviews = user_reviews[user_reviews['recommend'] == True].shape[0]
    
    if total_reviews == 0:
        percentage = 0
    
    percentage = (recommended_reviews / total_reviews) * 100
    
    items_per_user = expanded_df_items[expanded_df_items['user_id'] == User_id]['item_id'].count()
    
    return money_spent_by_user, percentage,items_per_user

In [35]:
#Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos 
#en base a reviews.recommend.
#( YYYY-MM-DD y YYYY-MM-DD : str )
def countreviews(start_date: str, end_date: str):
    filtered_reviews = expanded_df_reviews[(expanded_df_reviews['posted'] >= start_date) & (expanded_df_reviews['posted'] <= end_date)]
    unique_users= filtered_reviews['user_id'].nunique()
    
    recommended_reviews = filtered_reviews[filtered_reviews['recommend'] == True].shape[0]
    total_reviews = filtered_reviews.shape[0]
    
    if total_reviews == 0:
        recommendation_percentage = 0
    else:
        recommendation_percentage = (recommended_reviews / total_reviews) * 100
    return unique_users, recommendation_percentage

In [36]:
def genre(genre_name: str):
    #Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.
    genre_items = df_steam_games[df_steam_games['genres'].apply(lambda genres: genre_name in genres)]
    ranking = 0
    return genre_items

In [37]:
genre_name='Action'
genre_items = df_steam_games[df_steam_games['genres'].apply(lambda genres: genre_name in genres)]
genre_item_ids = genre_items['genres'].tolist()

valores_unicos = set()

for sublista in genre_item_ids:
    for valor in sublista:
        valores_unicos.add(valor)
genre_ranking = genre_items.merge(expanded_df_items, left_on='id', right_on='item_id')

def sumar_playtime(row):
    if row['genres'] in valores_unicos:
        return row['playtime_forever']
    else:
        return 0





In [38]:

def developer(desarrollador: str):
    # Filtrar los juegos del desarrollador
    juegos_del_desarrollador = df_steam_games[df_steam_games['publisher'] == desarrollador]
    
    # Extraer el año de lanzamiento de cada juego
    juegos_del_desarrollador['release_year'] = juegos_del_desarrollador['release_date'].str.extract(r'(\d{4})').astype(int)
    
    # Contar la cantidad de ítems por año utilizando .loc
    item_count_by_year = juegos_del_desarrollador.groupby('release_year').size().sort_index()

    # Calcular el porcentaje de contenido gratuito ("Free") por año
    free_items_by_year = juegos_del_desarrollador[juegos_del_desarrollador['price'] == 0]
    free_item_count_by_year = free_items_by_year.groupby('release_year').size().sort_index()
    total_item_count_by_year = item_count_by_year.reindex(free_item_count_by_year.index).fillna(0)
    percentage_free_by_year = (free_item_count_by_year / total_item_count_by_year) * 100
    
    return percentage_free_by_year


In [39]:
def sentiment_analysis(year: int):
    # Extraer el año de lanzamiento de cada juego
    #expanded_df_reviews['post_year'] = expanded_df_reviews['posted'].dt.year
    expanded_df_reviews['post_year'] = expanded_df_reviews['posted'].dt.year
    df_reviews_by_year = expanded_df_reviews[expanded_df_reviews['post_year'] == year]
    sentiment_counts = df_reviews_by_year['sentiment_analysis'].value_counts()
    # Crear un diccionario para almacenar los resultados
    result = sentiment_counts.to_dict()
    return result

<h1>Exportación de df limpios </h1>

In [42]:
expanded_df_items.to_parquet(r'clean_items.parquet.gzip', compression='gzip')
expanded_df_reviews.to_parquet(r'clean_reviews.parquet.gzip', compression='gzip')
df_steam_games.to_parquet(r'clean_games.parquet.gzip', compression='gzip')